# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import pandas as pd
import numpy as np

In [3]:
# load data from database
from sqlalchemy import create_engine

engine = create_engine('sqlite:///Disaster_Response_Data.db')
df = pd.read_sql("SELECT * FROM Disaster_Response_Data", engine)


In [4]:
df.dtypes

id                         int64
message                   object
original                  object
genre                     object
related                    int64
request                    int64
offer                      int64
aid_related                int64
medical_help               int64
medical_products           int64
search_and_rescue          int64
security                   int64
military                   int64
child_alone                int64
water                      int64
food                       int64
shelter                    int64
clothing                   int64
money                      int64
missing_people             int64
refugees                   int64
death                      int64
other_aid                  int64
infrastructure_related     int64
transport                  int64
buildings                  int64
electricity                int64
tools                      int64
hospitals                  int64
shops                      int64
aid_center

In [5]:
X = df.message
y = df[df.columns[4:]]

### 2. Write a tokenization function to process your text data

In [6]:
import nltk
import re
nltk.download(['wordnet', 'punkt'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    
    tokens = word_tokenize(text)
    
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens


/opt/conda/lib/python3.6/site-packages/sklearn/utils/fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [13]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multioutput import MultiOutputClassifier

pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
        ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [9]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...or=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=None))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [11]:
#Evaluation of model with AdaBoost Classifier 

from sklearn.metrics import classification_report

y_pred = pipeline.predict(X_test)

for i,name in enumerate(y_test):
    print(name)
    print(classification_report(y_test[name], y_pred[:, i]))

related
              precision    recall  f1-score   support

           0       0.59      0.26      0.36      1541
           1       0.80      0.94      0.86      4955
           2       0.50      0.10      0.17        58

   micro avg       0.78      0.78      0.78      6554
   macro avg       0.63      0.44      0.47      6554
weighted avg       0.75      0.78      0.74      6554

request
              precision    recall  f1-score   support

           0       0.91      0.97      0.94      5443
           1       0.77      0.55      0.65      1111

   micro avg       0.90      0.90      0.90      6554
   macro avg       0.84      0.76      0.79      6554
weighted avg       0.89      0.90      0.89      6554

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6523
           1       0.10      0.03      0.05        31

   micro avg       0.99      0.99      0.99      6554
   macro avg       0.55      0.52      0.52      655

### 6. Improve your model
Use grid search to find better parameters. 

In [12]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f11d9205840>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
             learning_rate=1.0, n_estimators=50, random_state=None),
              n_jobs=None))],
 'vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_r

In [14]:
from sklearn.model_selection import GridSearchCV

parameters = {'clf__estimator__n_estimators': [25,100],
              'clf__estimator__learning_rate': [0.02, 2.0]
                }


In [15]:
cv = GridSearchCV(pipeline, param_grid=parameters, cv=None, n_jobs=-1, verbose=3)

cv.fit(X_train, y_train)


/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] clf__estimator__learning_rate=0.02, clf__estimator__n_estimators=25 
[CV]  clf__estimator__learning_rate=0.02, clf__estimator__n_estimators=25, score=0.18751907232224596, total= 1.1min
[CV] clf__estimator__learning_rate=0.02, clf__estimator__n_estimators=25 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  1.2min remaining:    0.0s


[CV]  clf__estimator__learning_rate=0.02, clf__estimator__n_estimators=25, score=0.1776014647543485, total= 1.1min
[CV] clf__estimator__learning_rate=0.02, clf__estimator__n_estimators=25 


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  2.5min remaining:    0.0s


[CV]  clf__estimator__learning_rate=0.02, clf__estimator__n_estimators=25, score=0.1864510222764724, total= 1.1min
[CV] clf__estimator__learning_rate=0.02, clf__estimator__n_estimators=100 
[CV]  clf__estimator__learning_rate=0.02, clf__estimator__n_estimators=100, score=0.21071101617332927, total= 3.8min
[CV] clf__estimator__learning_rate=0.02, clf__estimator__n_estimators=100 
[CV]  clf__estimator__learning_rate=0.02, clf__estimator__n_estimators=100, score=0.20826975892584682, total= 3.9min
[CV] clf__estimator__learning_rate=0.02, clf__estimator__n_estimators=100 
[CV]  clf__estimator__learning_rate=0.02, clf__estimator__n_estimators=100, score=0.2122368019530058, total= 3.8min
[CV] clf__estimator__learning_rate=2.0, clf__estimator__n_estimators=25 
[CV]  clf__estimator__learning_rate=2.0, clf__estimator__n_estimators=25, score=0.20353982300884957, total= 1.1min
[CV] clf__estimator__learning_rate=2.0, clf__estimator__n_estimators=25 
[CV]  clf__estimator__learning_rate=2.0, clf__est

[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed: 32.1min finished


GridSearchCV(cv=None, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...or=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'clf__estimator__n_estimators': [25, 100], 'clf__estimator__learning_rate': [0.02, 2.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [32]:
y_pred_tuned = cv.predict(X_test)

for i,name in enumerate(y_test):
    print(name)
    print(classification_report(y_test[name], y_pred_tuned[:, i]))

related
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1550
           1       0.76      1.00      0.86      4957
           2       0.00      0.00      0.00        47

   micro avg       0.76      0.76      0.76      6554
   macro avg       0.25      0.33      0.29      6554
weighted avg       0.57      0.76      0.65      6554

request
              precision    recall  f1-score   support

           0       0.86      0.99      0.92      5459
           1       0.87      0.22      0.36      1095

   micro avg       0.86      0.86      0.86      6554
   macro avg       0.87      0.61      0.64      6554
weighted avg       0.87      0.86      0.83      6554

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6522
           1       0.00      0.00      0.00        32

   micro avg       1.00      1.00      1.00      6554
   macro avg       0.50      0.50      0.50      655

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.91      1.00      0.95      5941
           1       0.81      0.05      0.09       613

   micro avg       0.91      0.91      0.91      6554
   macro avg       0.86      0.52      0.52      6554
weighted avg       0.90      0.91      0.87      6554

fire
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6492
           1       1.00      0.03      0.06        62

   micro avg       0.99      0.99      0.99      6554
   macro avg       1.00      0.52      0.53      6554
weighted avg       0.99      0.99      0.99      6554

earthquake
              precision    recall  f1-score   support

           0       0.97      0.99      0.98      5988
           1       0.89      0.66      0.76       566

   micro avg       0.96      0.96      0.96      6554
   macro avg       0.93      0.83      0.87      6554
weighted avg       0.96      0.96      0.96      6554

col

In [56]:
report = classification_report(y_test[name], y_pred_tuned[:, i], output_dict=True)
print(pd.DataFrame(report).transpose())

              f1-score  precision    recall  support
0             0.368915   0.545455  0.278710   1550.0
1             0.857917   0.798887  0.926367   4957.0
2             0.131148   0.285714  0.085106     47.0
micro avg     0.767165   0.767165  0.767165   6554.0
macro avg     0.452660   0.543352  0.430061   6554.0
weighted avg  0.737058   0.735271  0.767165   6554.0


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [64]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [63]:
#Model with Random Forest Classifier

pipeline_rf = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
        ])

pipeline_rf.fit(X_train, y_train)

y_pred_rf = pipeline_rf.predict(X_test)

for i,name in enumerate(y_test):
    print(name)
    print(classification_report(y_test[name], y_pred_rf[:, i]))

/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The de

related
              precision    recall  f1-score   support

           0       0.60      0.34      0.44      1550
           1       0.81      0.93      0.87      4957
           2       0.40      0.04      0.08        47

   micro avg       0.78      0.78      0.78      6554
   macro avg       0.61      0.44      0.46      6554
weighted avg       0.76      0.78      0.76      6554

request
              precision    recall  f1-score   support

           0       0.88      0.99      0.93      5459
           1       0.83      0.35      0.50      1095

   micro avg       0.88      0.88      0.88      6554
   macro avg       0.86      0.67      0.71      6554
weighted avg       0.87      0.88      0.86      6554

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6522
           1       0.00      0.00      0.00        32

   micro avg       1.00      1.00      1.00      6554
   macro avg       0.50      0.50      0.50      655

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.95      1.00      0.98      6258
           1       0.00      0.00      0.00       296

   micro avg       0.95      0.95      0.95      6554
   macro avg       0.48      0.50      0.49      6554
weighted avg       0.91      0.95      0.93      6554

weather_related
              precision    recall  f1-score   support

           0       0.84      0.96      0.89      4778
           1       0.83      0.49      0.61      1776

   micro avg       0.83      0.83      0.83      6554
   macro avg       0.83      0.73      0.75      6554
weighted avg       0.83      0.83      0.82      6554

floods
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      6015
           1       0.90      0.28      0.42       539

   micro avg       0.94      0.94      0.94      6554
   macro avg       0.92      0.64      0.70      6554
weighted avg       0.94      0.94      0.92      65

In [65]:
#Model with KNN Classifier

pipeline_knn = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(KNeighborsClassifier()))
        ])

pipeline_knn.fit(X_train, y_train)

y_pred_knn = pipeline_knn.predict(X_test)

for i,name in enumerate(y_test):
    print(name)
    print(classification_report(y_test[name], y_pred_knn[:, i]))

related
              precision    recall  f1-score   support

           0       0.59      0.40      0.47      1550
           1       0.82      0.91      0.87      4957
           2       0.70      0.15      0.25        47

   micro avg       0.79      0.79      0.79      6554
   macro avg       0.70      0.49      0.53      6554
weighted avg       0.77      0.79      0.77      6554

request
              precision    recall  f1-score   support

           0       0.92      0.94      0.93      5459
           1       0.65      0.58      0.61      1095

   micro avg       0.88      0.88      0.88      6554
   macro avg       0.78      0.76      0.77      6554
weighted avg       0.87      0.88      0.87      6554

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6522
           1       0.00      0.00      0.00        32

   micro avg       0.99      0.99      0.99      6554
   macro avg       0.50      0.50      0.50      655

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [72]:
# Comparison of models

report_ab_tuned = classification_report(y_test[name], y_pred_tuned[:, i], output_dict=True)
print('AdaBoost Classifier Tuned')
print(pd.DataFrame(report_ada_tuned).transpose())

report_rf = classification_report(y_test[name], y_pred_rf[:, i], output_dict=True)
print('\n Random Forest Classifier')
print(pd.DataFrame(report_rf).transpose())

report_knn = classification_report(y_test[name], y_pred_knn[:, i], output_dict=True)
print('\n KNN Classifier')
print(pd.DataFrame(report_knn).transpose())

AdaBoost Classifier Tuned
              f1-score  precision    recall  support
0             0.922076   0.887265  0.959729   5314.0
1             0.578690   0.734491  0.477419   1240.0
micro avg     0.868477   0.868477  0.868477   6554.0
macro avg     0.750383   0.810878  0.718574   6554.0
weighted avg  0.857108   0.858361  0.868477   6554.0

 Random Forest Classifier
              f1-score  precision    recall  support
0             0.914727   0.857566  0.980053   5314.0
1             0.435793   0.779626  0.302419   1240.0
micro avg     0.851846   0.851846  0.851846   6554.0
macro avg     0.675260   0.818596  0.641236   6554.0
weighted avg  0.824114   0.842820  0.851846   6554.0

 KNN Classifier
              f1-score  precision    recall  support
0             0.908342   0.884609  0.933384   5314.0
1             0.542295   0.626188  0.478226   1240.0
micro avg     0.847269   0.847269  0.847269   6554.0
macro avg     0.725319   0.755398  0.705805   6554.0
weighted avg  0.839087   0.83

### 9. Export your model as a pickle file

In [16]:
import pickle

pickle.dump(cv, open('classifier.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.